# Model Clients

AGNext provides the {py:mod}`autogen_core.components.models` module with a suite of built-in
model clients for using ChatCompletion API.
All model clients implement the {py:class}`autogen_core.components.models.ChatCompletionClient` protocol class.

## Built-in Model Clients

Currently there are two built-in model clients:
{py:class}`autogen_core.components.models.OpenAIChatCompletionClient` and
{py:class}`autogen_core.components.models.AzureOpenAIChatCompletionClient`.
Both clients are asynchronous.

To use the {py:class}`~autogen_core.components.models.OpenAIChatCompletionClient`, you need to provide the API key
either through the environment variable `OPENAI_API_KEY` or through the `api_key` argument.

In [16]:
from autogen_core.components.models import OpenAIChatCompletionClient, UserMessage

# Create an OpenAI model client.
model_client = OpenAIChatCompletionClient(
    model="gpt-4o",
    # api_key="sk-...", # Optional if you have an API key set in the environment.
)

You can call the {py:meth}`~autogen_core.components.models.OpenAIChatCompletionClient.create` method to create a
chat completion request, and await for an {py:class}`~autogen_core.components.models.CreateResult` object in return.

In [17]:
# Send a message list to the model and await the response.
messages = [
    UserMessage(content="What is the capital of France?", source="user"),
]
response = await model_client.create(messages=messages)

# Print the response
print(response.content)

The capital of France is Paris.


### Streaming Response

You can use the {py:meth}`~autogen_core.components.models.OpenAIChatCompletionClient.create_streaming` method to create a
chat completion request with streaming response.

In [24]:
messages = [
    UserMessage(content="Write a very short story about a dragon.", source="user"),
]

# Create a stream.
stream = model_client.create_stream(messages=messages)

# Iterate over the stream and print the responses.
print("Streamed responses:")
async for response in stream:  # type: ignore
    if isinstance(response, str):
        # A partial response is a string.
        print(response, flush=True, end="")
    else:
        # The last response is a CreateResult object with the complete message.
        print("\n\n------------\n")
        print("The complete response:", flush=True)
        print(response.content, flush=True)

Streamed responses:
In the heart of the Whispering Woods lived Ember, a small dragon with scales of shimmering gold. Unlike other dragons, Ember breathed not fire but music, each note a whisper of ancient songs. The villagers, initially fearful, soon realized her gift brought harmony and joy.

One night, as darkness threatened the land, Ember's melodies summoned the stars, casting a protective glow. The villagers danced beneath the celestial orchestra, their worries dissolving like morning mist.

From that day on, Ember's song became a nightly ritual, a promise that light and harmony would always prevail. The dragon of the Whispering Woods was a symbol of peace, her golden scales a testament to the magic of gentleness.

------------

The complete response:
In the heart of the Whispering Woods lived Ember, a small dragon with scales of shimmering gold. Unlike other dragons, Ember breathed not fire but music, each note a whisper of ancient songs. The villagers, initially fearful, soon re

```{note}
The last response in the streaming response is always the final response
of the type {py:class}`~autogen_core.components.models.CreateResult`.
```

### Azure OpenAI

To use the {py:class}`~autogen_core.components.models.AzureOpenAIChatCompletionClient`, you need to provide
the deployment id, Azure Cognitive Services endpoint, api version, and model capabilities.
For authentication, you can either provide an API key or an Azure Active Directory (AAD) token credential.
To use AAD authentication, you need to first install the `azure-identity` package.

In [11]:
# pip install azure-identity

The following code snippet shows how to use AAD authentication.
The identity used must be assigned the [**Cognitive Services OpenAI User**](https://learn.microsoft.com/en-us/azure/ai-services/openai/how-to/role-based-access-control#cognitive-services-openai-user) role.

In [15]:
from autogen_core.components.models import AzureOpenAIChatCompletionClient
from azure.identity import DefaultAzureCredential, get_bearer_token_provider

# Create the token provider
token_provider = get_bearer_token_provider(DefaultAzureCredential(), "https://cognitiveservices.azure.com/.default")

az_model_client = AzureOpenAIChatCompletionClient(
    model="{your-azure-deployment}",
    api_version="2024-06-01",
    azure_endpoint="https://{your-custom-endpoint}.openai.azure.com/",
    azure_ad_token_provider=token_provider,  # Optional if you choose key-based authentication.
    # api_key="sk-...", # For key-based authentication.
    model_capabilities={
        "vision": True,
        "function_calling": True,
        "json_output": True,
    },
)

```{note}
See [here](https://learn.microsoft.com/en-us/azure/ai-services/openai/how-to/managed-identity#chat-completions) for how to use the Azure client directly or for more info.
```

## Build Agent using Model Client

Let's create a simple AI agent that can respond to messages using the ChatCompletion API.

In [4]:
from dataclasses import dataclass

from autogen_core.application import SingleThreadedAgentRuntime
from autogen_core.base import MessageContext
from autogen_core.components import RoutedAgent, message_handler
from autogen_core.components.models import ChatCompletionClient, OpenAIChatCompletionClient, SystemMessage, UserMessage


@dataclass
class Message:
    content: str


class SimpleAgent(RoutedAgent):
    def __init__(self, model_client: ChatCompletionClient) -> None:
        super().__init__("A simple agent")
        self._system_messages = [SystemMessage("You are a helpful AI assistant.")]
        self._model_client = model_client

    @message_handler
    async def handle_user_message(self, message: Message, ctx: MessageContext) -> Message:
        # Prepare input to the chat completion model.
        user_message = UserMessage(content=message.content, source="user")
        response = await self._model_client.create(
            self._system_messages + [user_message], cancellation_token=ctx.cancellation_token
        )
        # Return with the model's response.
        assert isinstance(response.content, str)
        return Message(content=response.content)

The `SimpleAgent` class is a subclass of the
{py:class}`autogen_core.components.TypeRoutedAgent` class for the convenience of automatically routing messages to the appropriate handlers.
It has a single handler, `handle_user_message`, which handles message from the user. It uses the `ChatCompletionClient` to generate a response to the message.
It then returns the response to the user, following the direct communication model.

```{note}
The `cancellation_token` of the type {py:class}`autogen_core.base.CancellationToken` is used to cancel
asynchronous operations. It is linked to async calls inside the message handlers
and can be used by the caller to cancel the handlers.
```

In [25]:
# Create the runtime and register the agent.
from autogen_core.base import AgentId

runtime = SingleThreadedAgentRuntime()
await runtime.register(
    "simple-agent",
    lambda: SimpleAgent(
        OpenAIChatCompletionClient(
            model="gpt-4o-mini",
            # api_key="sk-...", # Optional if you have an OPENAI_API_KEY set in the environment.
        )
    ),
)
# Start the runtime processing messages.
runtime.start()
# Send a message to the agent and get the response.
message = Message("Hello, what are some fun things to do in Seattle?")
response = await runtime.send_message(message, AgentId("simple-agent", "default"))
print(response.content)
# Stop the runtime processing messages.
await runtime.stop()

Seattle offers a wide range of activities and attractions for visitors. Here are some fun things to do in the city:

1. **Space Needle**: Visit this iconic landmark for stunning panoramic views of the city and surrounding mountains.

2. **Pike Place Market**: Explore this historic market where you can shop for fresh produce, local crafts, and enjoy delicious street food. Don't miss the famous fish-throwing!

3. **Chihuly Garden and Glass**: Admire the breathtaking glass art installations by artist Dale Chihuly, both indoors and in the beautiful outdoor garden.

4. **Museum of Pop Culture (MoPOP)**: Discover exhibits focused on music, science fiction, and pop culture, including artifacts from famous films and music legends.

5. **Seattle Aquarium**: Learn about marine life native to the Pacific Northwest and see fascinating exhibits, including sea otters and jellyfish.

6. **Fremont Troll**: Take a photo with this quirky public art installation, a large troll sculpture located under the